In [12]:
import pandas as pd
df = pd.read_excel("https://www.thewindpower.net/data/samples/Sample_World_Windfarms.xls", sheet_name="Windfarms", skiprows=[1])
df.pop("2nd name")
df

,ID,Continent,ISO code,Country,State code,Area,City,Name,Latitude,Longitude,...,Number of turbines,Total power,Developer,Operator,Owner,Commissioning date,Status,Decommissioning date,Link,Update
0,27,Oceania,AU,Australia,QLD,Queensland,Ravenshoe,Windy Hill,-17.590006,145.531549,...,20,12000,Stanwell Corp,Stanwell Corp,Ratch Australia Corporation,2000/08,Production,NaN,Link,2022-10-25
1,13,Europe,BE,Belgium,#ND,Brabant wallon (Wallonie),Villers-la-Ville,Marbais,50.537381,4.514866,...,8,16000,Eneco,Electrastar SA,Eneco,2007,Production,NaN,Link,2022-10-20
2,12,Europe,BE,Belgium,#ND,Liège (Wallonie),Villers le Bouillet,Villers le Bouillet,50.580000,5.240000,...,6,9000,EDF renewables/Energie 2030,EDF renewables,EDF Luminus,2005/02,Production,NaN,Link,2022-05-20
3,16,Europe,BE,Belgium,#ND,Namur (Wallonie),Floreffe,Floreffe,50.422861,4.718278,...,1,2350,EDF Luminus,EDF Luminus,EDF Luminus,2010,Production,NaN,Link,2022-05-20
4,11,Europe,BE,Belgium,#ND,Namur (Wallonie),Walcourt,Tarcienne,50.302828,4.504480,...,6,9000,EDF renewables,EDF renewables,EDF Luminus,2005/11,Production,NaN,Link,2022-05-20
5,14,North America,CA,Canada,ON,Ontario,Shelburne,Melancthon,44.090000,-80.308000,...,45,67500,TransAlta,TransAlta,TransAlta,2006/03,Production,NaN,Link,2022-10-25
6,15,North America,CA,Canada,QC,Québec,Baie-des-Sables,Baie-des-Sables,48.710334,-67.869930,...,73,109500,Cartier Wind Energy,Innergex,Innergex/#ND,2006/11,Production,NaN,Link,2022-10-25
7,20,Europe,FR,France,#ND,11 (Occitanie),Roquetaillade,Roquetaillade,43.001132,2.233143,...,8,5280,Compagnie du Vent,Engie,Engie,2001/10,Production,NaN,Link,2022-08-17
8,22,Europe,FR,France,#ND,19 (Nouvelle-Aquitaine),Peyrelevade,Peyrelevade,45.760000,2.060000,...,6,9000,EED/Cegelec,MC3/Cohérence Energies,MC3,2005/01,Production,NaN,Link,2022-08-16
9,9,Europe,FR,France,#ND,26 (Auvergne-Rhône-Alpes),Donzère,Donzère,44.450000,4.750000,...,5,3000,Adelis/Valeco/Poweo,Valeco/NeoElectra,Valeco,1999/09,Production,NaN,Link,2022-08-01


In [13]:
df_power_curves = pd.read_excel("https://www.thewindpower.net/data/samples/Sample_Power_curves.xls", sheet_name="Power_curves", index_col=[0, 1, 2, 3])
df_power_curves = df_power_curves.transpose()
df_power_curves = df_power_curves[:-2]
df_power_curves.index = df_power_curves.iloc[:, 0]
df_power_curves.index.name = "Wind speed (m/s)"
df_power_curves = df_power_curves.iloc[:, 1:]
column_names = df_power_curves.columns.names
df_power_curves.columns = pd.MultiIndex.from_tuples(
    [(int(x[0]), x[1], int(x[2]), x[3]) for x in df_power_curves.columns.to_list()],
    names=column_names
)
df_power_curves

Man. ID,3,4,8,9,14
Manufucturer Name,Enercon,Gamesa,Nordex,Repower,Vestas
Turb. ID,495,81,7,16,413
Turbine Name,E82/2300,G128/4500,N90/2500,MM82,V112/3000
Wind speed (m/s),,,,,
0.0,0,0,0,0,0
0.5,0.0,0.0,0.0,0.0,0.0
1.0,0,0,0,0,0
1.5,0.0,0.0,0.0,0.0,0.0
2.0,3,75,0,0,0
...,...,...,...,...,...


In [14]:
import xarray as xr
ds = xr.open_dataset("/shared1/ICS/LOT1/windpower/ID_14_S200001010000_E200212312300_LAT44.0_LON99.75.nc")

ds

<xarray.Dataset>
Dimensions:    (time: 26304)
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01 ... 2002-12-31T23:00:00
    longitude  float32 ...
    latitude   float32 ...
Data variables:
    ws100      (time) float32 ...
Attributes:
    CDI:                 Climate Data Interface version 1.9.6 (http://mpimet....
    Conventions:         CF-1.6
    history:             Mon Mar 20 15:39:46 2023: cdo -b F32 chname,u100,ws1...
    CDO:                 Climate Data Operators version 1.9.6 (http://mpimet....
    bias_adj_procedure:  Delta method: https://www.nature.com/articles/s41597...
    reference dataset:   Global Wind Atlas 3.0, a free, web-based application...

In [43]:
import numpy as np

def power_conversion_function(wind_speed):
    out_da = xr.zeros_like(wind_speed)
    for t in wind_speed.time.values:
        nearest_wind_speed = df_power_curves.index[np.abs(df_power_curves.index - wind_speed.sel(time=t).values).values.astype(np.float32).argmin()]
        out_da.loc[dict(time=t)] = df_power_curves.loc[nearest_wind_speed].iloc[0]
    return out_da

ds.ws100
power_da = power_conversion_function(ds.ws100)
power_da

<xarray.DataArray 'ws100' (time: 26304)>
array([1180.,  998.,  815., ..., 1180., 1180., 1180.], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01 ... 2002-12-31T23:00:00
    longitude  float32 ...
    latitude   float32 ...
Attributes:
    long_name:  100 metre wind speed
    units:      m s**-1

In [42]:
np.abs(df_power_curves.index - ds.ws100.isel(time=0).values).values.astype(np.float32).argmin()

18